In [1]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from keras.datasets import cifar10


def prepare_data():


# データを用意する

# returns:

# x_train(ndarray)
# 訓練データ (5000,32 ,32, 3)

# x_test(ndarray)
# テストデータ (1000,32 ,32, 3)

# y_train(ndarray)
# 訓練データ(正解値)のone-hot化した正解ラベル(50000,)

# y_test(ndarray)
# テストデータ(正解値)のone-hot化した正解ラベル(10000,)



    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    # 訓練用とテスト用の画像データを正規化する
    x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
    x_train, x_test = x_train/255.0, x_test/255.0

    # 訓練用とテスト用の正解ラベルを10クラスのOne-Hotベクトルに変換
    y_train, y_test = to_categorical(y_train), to_categorical(y_test)

    return x_train, x_test, y_train, y_test

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense     # core layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D        # convolution layers
from tensorflow.keras import optimizers


def define_model():
    #Sequentialオブジェクトを生成
    model = Sequential()

#  第1層：畳み込み層1
#  (バッチサイズ 32,32,3) -> (バッチサイズ 32,32,32)
    model.add(
        Conv2D(
              filters = 32,                   #フィルター数は32
              kernel_size=(3,3),              # 3x3 のフィルターを使用
              input_shape= x_train[0].shape,   #
              padding='same',                 # ゼロパディングを行う
              activation='relu'               # 活性化関数はReLU
              #kernel_initializer='he_uniform', 初期化の方法
              ))

 #  第2層：畳み込み層2
 #  (バッチサイズ 32,32,32) -> (バッチサイズ 32,32,32)
    model.add(
        Conv2D(
              filters = 32,       #フィルター数は32
              kernel_size=(3,3),  # 3x3 のフィルターを使用
              activation='relu',  # 活性化関数はReLU
              padding='same',     # ゼロパディングを行う
              ))

 #  第3層：プーリング層1
 #  (バッチサイズ 32,32,32) -> (バッチサイズ 16,16,32)
    model.add(MaxPooling2D(pool_size=(2,2)))    # 2 x 2 の範囲ごとに最大値を抜き出す (Max Pooling)
    model.add(Dropout(0.5))                     # ドロップアウトは50％ (ドロップアウト１)

 #  第4層：畳み込み層3
 #  (バッチサイズ 16,16,32) -> (バッチサイズ 16,16,64)
    model.add(
          Conv2D(
                filters = 64,       #フィルター数は64
                kernel_size=(3,3),  # 3x3 のフィルターを使用
                activation='relu',  # 活性化関数はReLU
                padding='same',     # ゼロパディングを行う
                ))

 #  第5層：畳み込み層4
 #  (バッチサイズ 16,16,64) -> (バッチサイズ 16,16,64)
    model.add(
          Conv2D(
                filters = 64,       #フィルター数は64
                kernel_size=(3,3),  # 3x3 のフィルターを使用
                activation='relu',  # 活性化関数はReLU
                padding='same',     # ゼロパディングを行う
                ))

 #  第6層：プーリング層2
 #  (バッチサイズ 16,16,64) -> (バッチサイズ 8,8,64)
    model.add(MaxPooling2D(pool_size=(2,2)))    # 2 x 2 の範囲ごとに最大値を抜き出す (Max Pooling)
    model.add(Dropout(0.5))                     # ドロップアウトは50％ (ドロップアウト２)

 # Flatten
 # (バッチサイズ 8,8,64) -> (バッチサイズ ,4096)
    model.add(Flatten())

 #  第7層：全結合層
 # (バッチサイズ ,4096) -> (バッチサイズ ,512)
    model.add(Dense(512, activation='relu'))    # ニューロンの数は512, 活性化関数はReLU

 # (ドロップアウト3) : ドロップアウトは50％
    model.add(Dropout(0.5))

 #  第8層： 出力層
 # (バッチサイズ ,512) -> (バッチサイズ ,10)
    model.add(Dense(10, activation='softmax'))    # ニューロンの数は10, 活性化関数はソフトマックス

    model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizers.RMSprop(learning_rate=0.0001),
        metrics=['accuracy']
    )

    return model

In [3]:
#3. 訓練を実施する関数

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau

def train(x_train, x_test, y_train, y_test):

    #val_accuracy(検証データの精度)の改善が5エポック(学習回数)の中で見られなかったら、学習率を0.5倍する。
    reduce_lr = ReduceLROnPlateau(
        monitor='val_accuracy',   # 監視対象は検証データの精度
        factor=0.5,     # 学習率を減衰させる割合
        patience=5,     # 監視対象のエポック数
        verbose=1,
        mode='max',      # 最高値を監視する
        min_lr=0.00001  # 学習率の下限
    )

    model = define_model()
    model.summary()

    callbacks_list = [reduce_lr]

    # データ拡張
    datagen = ImageDataGenerator(
        width_shift_range=0.1,      # 横サイズの0.1の割合でランダムに水平移動
        height_shift_range=0.1,     # 縦サイズの0.1の割合でランダムに垂直移動
        rotation_range=10,          # 10度の範囲でランダムに回転
        zoom_range=0.1,             # ランダムに拡大
        horizontal_flip=True        # 左右反転
    )


    # ミニバッチのサイズ
    batch_size = 64

    # 学習回数
    epochs = 120

    # 学習を行う
    history = model.fit(
        x_train, y_train,
        batch_size = batch_size,
        epochs=epochs,
        verbose=1,
        validation_split=0.2,
        shuffle=True,
        callbacks=callbacks_list
    )

    #
    score = model.evaluate(x_test, y_test, verbose=0)

    # 損失関数を用いてlossを出力
    print('Test loss:', score[0])

    # テストデータの精度を出力
    print('Test accuracy:', score[1])

    return history

In [4]:
#
x_train, x_test, y_train, y_test = prepare_data()
history = train(x_train, x_test, y_train, y_test)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     2,097,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,168,362 (8.27 MB)

 Trainable params: 2,168,362 (8.27 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/120
625/625 ━━━━━━━━━━━━━━━━━━━━ 250s 396ms/step - accuracy: 0.1725 - loss: 2.2002 - val_accuracy: 0.3624 - val_loss: 1.8423 - learning_rate: 1.0000e-04
Epoch 2/120
625/625 ━━━━━━━━━━━━━━━━━━━━ 255s 386ms/step - accuracy: 0.3597 - loss: 1.7875 - val_accuracy: 0.4034 - val_loss: 1.6874 - learning_rate: 1.0000e-04
Epoch 3/120
625/625 ━━━━━━━━━━━━━━━━━━━━ 239s 382ms/step - accuracy: 0.4051 - loss: 1.6379 - val_accuracy: 0.4495 - val_loss: 1.5423 - learning_rate: 1.0000e-04
Epoch 4/120
625/625 ━━━━━━━━━━━━━━━━━━━━ 245s 392ms/step - accuracy: 0.4426 - loss: 1.5432 - val_accuracy: 0.4714 - val_loss: 1.4766 - learning_rate: 1.0000e-04
Epoch 5/120
625/625 ━━━━━━━━━━━━━━━━━━━━ 263s 394ms/step - accuracy: 0.4716 - loss: 1.4618 - val_accuracy: 0.5192 - val_loss: 1.3543 - learning_rate: 1.0000e-04
Epoch 6/120
625/625 ━━━━━━━━━━━━━━━━━━━━ 239s 383ms/step - accuracy: 0.4969 - loss: 1.4050 - val_accuracy: 0.5001 - val_loss: 1.3873 - learning_rate: 1.0000e-04
Epoch 7/120
625/625 ━━━━━━━━━━━━━━